# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [113]:
%%time
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option("display.max_rows", 500, "display.max_columns", 500)
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

Wall time: 531 ms
Parser   : 327 ms


In [114]:
data = pd.read_csv('..\..\..\IronHack\8.04 Natural Language Processing\Sentiment140.csv')

In [115]:
data.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [116]:
data.columns = ['target','id','date','flag','user','text']
data.columns

Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')

In [117]:
# data_sample = data.head(10000) due to the order, my accuracy score is 1 so I will use sample instead
data_sample = data.sample(n = 20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [118]:
def clean_up(s):
    
    return re.sub('[^a-z]', ' ', re.sub('http[s]?://[/\w\.]*', '', s.lower())).strip()

In [119]:
sample_clean = data_sample.text.apply(clean_up)

In [120]:
def tokenize(s):
    s = word_tokenize(s)
    s = [word for word in s if word.isalnum()]
    return s

In [121]:
sample_tokenized = sample_clean.apply(tokenize)

In [122]:
def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

In [123]:
sample_stemlem = sample_tokenized.apply(stem_and_lemmatize)

In [124]:
def remove_stopwords(l):
    l = [word for word in l if not word in stopwords.words('english')]
    return l

In [125]:
sample_stopwords = sample_stemlem.apply(remove_stopwords)
sample_stopwords.head()

917317          [sageeb, well, onli, find, whi, first, know]
728535     [believ, feel, sad, rip, br, ceci, stop, cri, ...
1168000                              [becirothon, oooh, see]
1254903                                         [haha, cool]
1412687    [kombiz, dang, good, work, save, coupl, get, b...
Name: text, dtype: object

In [126]:
data_sample['text_processed'] = sample_stopwords
data_sample

,target,id,date,flag,user,text,text_processed
917317,4,1753467232,Sun May 10 01:47:05 PDT 2009,NO_QUERY,hilzfuld,@sageeb Well you can only find out WHY by firs...,"[sageeb, well, onli, find, whi, first, know]"
728535,0,2263090719,Sat Jun 20 23:55:58 PDT 2009,NO_QUERY,allivsamson,I can't believe it! I feel so sad. RIP Br. Cec...,"[believ, feel, sad, rip, br, ceci, stop, cri, ..."
1168000,4,1980109558,Sun May 31 05:43:08 PDT 2009,NO_QUERY,getfrank,@BeciRothon oooh see you there!,"[becirothon, oooh, see]"
1254903,4,1997179664,Mon Jun 01 16:23:40 PDT 2009,NO_QUERY,Kelsieee,http://bit.ly/19HHwZ haha ain't that cool,"[haha, cool]"
1412687,4,2056700019,Sat Jun 06 11:37:48 PDT 2009,NO_QUERY,ggreeneva,@kombiz dang -- good work. Save a couple for w...,"[kombiz, dang, good, work, save, coupl, get, b..."
...,...,...,...,...,...,...,...
1304967,4,2010892450,Tue Jun 02 18:08:23 PDT 2009,NO_QUERY,ruipmarinho,TF-IDF algorithm implemented with success tom...,"[tf, idf, algorithm, implement, success, tomor..."
888478,4,1687307873,Sun May 03 08:20:42 PDT 2009,NO_QUERY,Huddylicious,@notaplayer83 Can you PM me the link too? Pret...,"[notaplay, pm, link, pretti, pretti, pleaaaaaa..."
119783,0,1831145059,Sun May 17 18:18:09 PDT 2009,NO_QUERY,DeannaSmith,@rascalkpr Sorry Wish u guys lived down here.,"[rascalkpr, sorri, wish, u, guy, live]"
261897,0,1986039434,Sun May 31 18:13:31 PDT 2009,NO_QUERY,davilivingston,@ carymac work,"[carymac, work]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [127]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [128]:
sample_processed = flatten(data_sample['text_processed'])
freqdist = FreqDist(sample_processed).most_common(5000)
pd.DataFrame(freqdist).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,4750,4751,4752,4753,4754,4755,4756,4757,4758,4759,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808,4809,4810,4811,4812,4813,4814,4815,4816,4817,4818,4819,4820,4821,4822,4823,4824,4825,4826,4827,4828,4829,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839,4840,4841,4842,4843,4844,4845,4846,4847,4848,4849,4850,4851,4852,4853,4854,4855,4856,4857,4858,4859,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869,4870,4871,4872,4873,4874,4875,4876,4877,4878,4879,4880,4881,4882,4883,4884,4885,4886,4887,4888,4889,4890,4891,4892,4893,4894,4895,4896,4897,4898,4899,4900,4901,4902,4903,4904,4905,4906,4907,4908,4909,4910,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920,4921,4922,4923,4924,4925,4926,4927,4928,4929,4930,4931,4932,4933,4934,4935,4936,4937,4938,4939,4940,4941,4942,4943,4944,4945,4946,4947,4948,4949,4950,4951,4952,4953,4954,4955,4956,4957,4958,4959,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,go,day,wa,get,good,thi,work,like,love,quot,u,got,today,time,lol,one,thank,miss,know,want,back,feel,amp,im,think,realli,night,see,hope,new,watch,make,na,well,need,oh,still,home,come,ha,look,twitter,last,much,morn,tomorrow,wish,great,sleep,haha,wait,would,tri,week,whi,sad,right,fun,happi,follow,thing,tonight,onli,bad,sorri,way,veri,friend,gon,say,hi,nice,tweet,better,yeah,show,hate,take,start,though,hour,school,bed,peopl,even,could,hey,let,ye,final,tire,lt,use,littl,best,guy,dont,weekend,rain,girl,suck,awesom,plea,x,play,call,movi,cant,ani,wan,everyon,long,never,someth,done,next,year,sure,soon,life,ok,first,leav,find,sick,keep,song,alway,alreadi,made,eat,live,hous,n,head,phone,lot,b,cool,enjoy,readi,help,bore,talk,pretti,man,befor,w,ur,excit,hurt,big,anoth,ever,thought,mayb,read,tell,damn,went,away,finish,someon,becaus,babi,yet,happen,earli,pic,listen,game,guess,weather,hot,yay,wonder,summer,omg,said,ta,amaz,sound,also,mom,mean,doe,lost,ugh,end,exam,bit,around,two,birthday,put,music,check,run,sunday,poor,late,car,hear,sinc,post,old,stop,left,yesterday,meet,cri,sun,ya,updat,parti,th,give,may,stay,world,stuff,later,noth,buy,hair,r,monday,month,beauti,hard,friday,glad,job,actual,wow,aww,com,anyon,dinner,cold,boy,saw,gone,kid,fuck,luck,die,p,shit,tv,...,sudah,dodgi,seminar,jacob,dem,soho,fwd,tad,jab,owwww,psychic,deadli,shouldv,herbal,jar,murphi,ascot,bugger,tm,crackin,inbox,cucumb,grandmoth,adriana,valv,mwah,fearless,rod,az,sleeep,sweetz,pork,globe,factor,coma,cock,nichol,denis,marginatasnaili,twitterberri,grrrrr,maria,formula,reader,bubba,juri,sickest,cg,instrument,frankiethesat,candl,ooooo,snooz,layer,silver,welp,newspap,ide,bladder,scoobi,verizon,taipei,upcom,tranc,flo,sama,betti,poland,unavail,mhelp,pdf,buddhist,heartbroken,tweak,drinkin,chap,poser,outing,dah,claw,buffi,cynic,bk,alexi,nil,waffl,janin,heavenli,discount,bra,

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [129]:
freq_df = pd.DataFrame(freqdist).T
freq_df.iloc[0]

0                  go
1                 day
2                  wa
3                 get
4                good
            ...      
4995            reali
4996            blahh
4997          sqlfool
4998          smaller
4999    misunderstand
Name: 0, Length: 5000, dtype: object

In [130]:
freq_df.columns = freq_df.iloc[0]

In [131]:
freq_df

,go,day,wa,get,good,thi,work,like,love,quot,u,got,today,time,lol,one,thank,miss,know,want,back,feel,amp,im,think,realli,night,see,hope,new,watch,make,na,well,need,oh,still,home,come,ha,look,twitter,last,much,morn,tomorrow,wish,great,sleep,haha,wait,would,tri,week,whi,sad,right,fun,happi,follow,thing,tonight,onli,bad,sorri,way,veri,friend,gon,say,hi,nice,tweet,better,yeah,show,hate,take,start,though,hour,school,bed,peopl,even,could,hey,let,ye,final,tire,lt,use,littl,best,guy,dont,weekend,rain,girl,suck,awesom,plea,x,play,call,movi,cant,ani,wan,everyon,long,never,someth,done,next,year,sure,soon,life,ok,first,leav,find,sick,keep,song,alway,alreadi,made,eat,live,hous,n,head,phone,lot,b,cool,enjoy,readi,help,bore,talk,pretti,man,befor,w,ur,excit,hurt,big,anoth,ever,thought,mayb,read,tell,damn,went,away,finish,someon,becaus,babi,yet,happen,earli,pic,listen,game,guess,weather,hot,yay,wonder,summer,omg,said,ta,amaz,sound,also,mom,mean,doe,lost,ugh,end,exam,bit,around,two,birthday,put,music,check,run,sunday,poor,late,car,hear,sinc,post,old,stop,left,yesterday,meet,cri,sun,ya,updat,parti,th,give,may,stay,world,stuff,later,noth,buy,hair,r,monday,month,beauti,hard,friday,glad,job,actual,wow,aww,com,anyon,dinner,cold,boy,saw,gone,kid,fuck,luck,die,p,shit,tv,...,sudah,dodgi,seminar,jacob,dem,soho,fwd,tad,jab,owwww,psychic,deadli,shouldv,herbal,jar,murphi,ascot,bugger,tm,crackin,inbox,cucumb,grandmoth,adriana,valv,mwah,fearless,rod,az,sleeep,sweetz,pork,globe,factor,coma,cock,nichol,denis,marginatasnaili,twitterberri,grrrrr,maria,formula,reader,bubba,juri,sickest,cg,instrument,frankiethesat,candl,ooooo,snooz,layer,silver,welp,newspap,ide,bladder,scoobi,verizon,taipei,upcom,tranc,flo,sama,betti,poland,unavail,mhelp,pdf,buddhist,heartbroken,tweak,drinkin,chap,poser,outing,dah,claw,buffi,cynic,bk,alexi,nil,waffl,janin,heavenli,discount,bra,jimmycarr,mcintyr,liquid,plzzz,recruit,matur,elys,geezer,christma,claim,vamano,anoop,championship,careless,alik,salon,oldi,sunda,tilt,furri,te,grove,andyhurleyday,awwwwwww,froze,reset,oooooh,vice,deer,urgent,agoni,jonathan,tournament,happili,chart,incom,bitchi,foundat,hbo,naomi,drawer,flea,qt,thisisdavina,downpour,rocki,babysitt,violin,chemo,keynot,grew,epiphanygirl,davi,kodi,op,leo,fountain,someday,mentor,innoc,drool,gcse,sacr,colleagu,quarantin,concern,yumm,ahoy,musicmonday,extern,sculli,copyright,twittertakeov,xoxoxo,cobra,starship,stub,thot,wi,shmephani,peru,permit,twix,gloriou,insecur,pd,moi,db,mrw,dre,disconnect,humor,ahaa,guilt,karma,ooc,mikegentil,nemo,crystalchappel,stuart,champion,journalist,courtney,zzzzz,rin,assembl,thesi,croydon,azz,goof,schofe,dragon,reflect,jennif,gel,hub,liver,wakeup,borrow,thailand,substitut,enhoy,meatbal,skirt,liam,anticip,bridg,dum,noe,penelop,giveaway,colin,beckybuckwild,eurgh,embarass,orient,bermuda,penguin,blanki,mystic,psu,limbeck,honk,stellar,tcherryx,headlight,luke,fought,pvc,greggarbo,gage,adi,jwage,spice,tram,reali,blahh,sqlfool,smaller,misunderstand
0,go,day,wa,get,good,thi,work,like,love,quot,u,got,today,time,lol,one,thank,miss,know,want,back,feel,amp,im,think,realli,night,see,hope,new,watch,make,na,well,need,oh,still,home,come,ha,look,twitter,last,much,morn,tomorrow,wish,great,sleep,haha,wait,would,tri,week,whi,sad,right,fun,happi,follow,thing,tonight,onli,bad,sorri,way,veri,friend,gon,say,hi,nice,tweet,better,yeah,show,hate,take,start,though,hour,school,bed,peopl,even,could,hey,let,ye,final,tire,lt,use,littl,best,guy,dont,weekend,rain,girl,suck,awesom,plea,x,play,call,movi,cant,ani,wan,everyon,long,never,someth,done,next,year,sure,soon,life,ok,first,leav,find,sick,keep,song,alway,alreadi,made,eat,live,hous,n,head,phone,lot,b,cool,enjoy,readi,help,bore,talk,pretti,man,befor,w,ur,excit,hurt,big,anoth,ever,thought,mayb,read,tell,damn,went,away,finish,someon,becaus,babi,yet,happen,earli,pic,listen,game,guess,weather,hot,yay,wonder,summer,omg,said,ta,amaz,sound,also,mom,mean,doe,lost,ugh,end,exam,bit,around,two,birthday,put,music,check,run,sunday,poor,late,car,hear,s

In [132]:
freq_df.drop([0], axis = 0, inplace = True) 

In [133]:
freq_df

,go,day,wa,get,good,thi,work,like,love,quot,u,got,today,time,lol,one,thank,miss,know,want,back,feel,amp,im,think,realli,night,see,hope,new,watch,make,na,well,need,oh,still,home,come,ha,look,twitter,last,much,morn,tomorrow,wish,great,sleep,haha,wait,would,tri,week,whi,sad,right,fun,happi,follow,thing,tonight,onli,bad,sorri,way,veri,friend,gon,say,hi,nice,tweet,better,yeah,show,hate,take,start,though,hour,school,bed,peopl,even,could,hey,let,ye,final,tire,lt,use,littl,best,guy,dont,weekend,rain,girl,suck,awesom,plea,x,play,call,movi,cant,ani,wan,everyon,long,never,someth,done,next,year,sure,soon,life,ok,first,leav,find,sick,keep,song,alway,alreadi,made,eat,live,hous,n,head,phone,lot,b,cool,enjoy,readi,help,bore,talk,pretti,man,befor,w,ur,excit,hurt,big,anoth,ever,thought,mayb,read,tell,damn,went,away,finish,someon,becaus,babi,yet,happen,earli,pic,listen,game,guess,weather,hot,yay,wonder,summer,omg,said,ta,amaz,sound,also,mom,mean,doe,lost,ugh,end,exam,bit,around,two,birthday,put,music,check,run,sunday,poor,late,car,hear,sinc,post,old,stop,left,yesterday,meet,cri,sun,ya,updat,parti,th,give,may,stay,world,stuff,later,noth,buy,hair,r,monday,month,beauti,hard,friday,glad,job,actual,wow,aww,com,anyon,dinner,cold,boy,saw,gone,kid,fuck,luck,die,p,shit,tv,...,sudah,dodgi,seminar,jacob,dem,soho,fwd,tad,jab,owwww,psychic,deadli,shouldv,herbal,jar,murphi,ascot,bugger,tm,crackin,inbox,cucumb,grandmoth,adriana,valv,mwah,fearless,rod,az,sleeep,sweetz,pork,globe,factor,coma,cock,nichol,denis,marginatasnaili,twitterberri,grrrrr,maria,formula,reader,bubba,juri,sickest,cg,instrument,frankiethesat,candl,ooooo,snooz,layer,silver,welp,newspap,ide,bladder,scoobi,verizon,taipei,upcom,tranc,flo,sama,betti,poland,unavail,mhelp,pdf,buddhist,heartbroken,tweak,drinkin,chap,poser,outing,dah,claw,buffi,cynic,bk,alexi,nil,waffl,janin,heavenli,discount,bra,jimmycarr,mcintyr,liquid,plzzz,recruit,matur,elys,geezer,christma,claim,vamano,anoop,championship,careless,alik,salon,oldi,sunda,tilt,furri,te,grove,andyhurleyday,awwwwwww,froze,reset,oooooh,vice,deer,urgent,agoni,jonathan,tournament,happili,chart,incom,bitchi,foundat,hbo,naomi,drawer,flea,qt,thisisdavina,downpour,rocki,babysitt,violin,chemo,keynot,grew,epiphanygirl,davi,kodi,op,leo,fountain,someday,mentor,innoc,drool,gcse,sacr,colleagu,quarantin,concern,yumm,ahoy,musicmonday,extern,sculli,copyright,twittertakeov,xoxoxo,cobra,starship,stub,thot,wi,shmephani,peru,permit,twix,gloriou,insecur,pd,moi,db,mrw,dre,disconnect,humor,ahaa,guilt,karma,ooc,mikegentil,nemo,crystalchappel,stuart,champion,journalist,courtney,zzzzz,rin,assembl,thesi,croydon,azz,goof,schofe,dragon,reflect,jennif,gel,hub,liver,wakeup,borrow,thailand,substitut,enhoy,meatbal,skirt,liam,anticip,bridg,dum,noe,penelop,giveaway,colin,beckybuckwild,eurgh,embarass,orient,bermuda,penguin,blanki,mystic,psu,limbeck,honk,stellar,tcherryx,headlight,luke,fought,pvc,greggarbo,gage,adi,jwage,spice,tram,reali,blahh,sqlfool,smaller,misunderstand
1,1704,1428,1372,1342,1162,1145,1095,1054,1049,917,895,888,855,823,773,767,732,723,704,702,685,663,638,632,624,613,589,588,572,556,552,547,540,529,529,520,509,507,484,481,477,468,466,454,444,428,428,424,408,401,391,389,376,368,366,362,360,353,353,350,344,344,340,340,335,331,327,323,318,317,306,306,305,294,290,290,285,285,284,283,275,274,267,263,263,262,260,253,244,243,242,241,240,237,237,236,236,231,231,230,227,226,223,222,221,219,215,215,214,213,212,211,209,206,205,205,204,204,204,202,200,198,197,194,192,191,190,189,189,188,187,185,182,178,177,174,174,174,173,173,171,171,171,170,169,169,169,169,169,168,167,165,164,164,163,163,163,161,160,158,156,156,155,154,153,153,153,152,152,150,149,148,147,146,146,145,144,144,144,143,143,143,142,142,141,139,139,139,139,139,135,135,134,134,134,133,133,132,130,130,130,130,129,129,129,129,128,128,128,128,127,126,126,125,125,125,124,124,123,123,122,122,122,121,121,121,121,121,121,120,120,118,118,117,117,117,117,115,115,115,114,114,114,114,113,111,111,110,110,108,...,3,3,3,3,3,3,3,3,3,3,

In [134]:
def find_features(document):
    words = set(document)
    return {w:(w in words) for w in freq_df.columns}

In [135]:
documents = list(zip(data_sample['text_processed'], np.where(data_sample['target'] == 4, True, False)))
#documents

In [136]:
%%time
featuresets = [(find_features(words), category) for (words, category) in documents]

Wall time: 19.7 s


In [137]:
#featuresets

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [138]:
train_set, test_set = featuresets[:500], featuresets[500:]

In [139]:
%%time
classifier = nltk.NaiveBayesClassifier.train(train_set)

Wall time: 5.12 s
Parser   : 199 ms


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [140]:
%%time
nltk.classify.accuracy(classifier, test_set)

Wall time: 4min 19s
Parser   : 878 ms


0.6273846153846154

In [141]:
%%time
classifier.show_most_informative_features(15)

Most Informative Features
                    miss = True            False : True   =      6.6 : 1.0
                   still = True            False : True   =      6.3 : 1.0
                    haha = True             True : False  =      5.0 : 1.0
                       c = True             True : False  =      4.3 : 1.0
                    damn = True            False : True   =      4.3 : 1.0
                    girl = True             True : False  =      4.3 : 1.0
                  wonder = True             True : False  =      4.3 : 1.0
                    wish = True            False : True   =      3.9 : 1.0
                     fun = True             True : False  =      3.8 : 1.0
                     hey = True             True : False  =      3.8 : 1.0
                     tri = True            False : True   =      3.8 : 1.0
                   anoth = True             True : False  =      3.7 : 1.0
                     buy = True            False : True   =      3.7 : 1.0

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [142]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [143]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [144]:
# your code here